####Exercise: Use downloaded embeddings (either Glove or word2vec) to build a model to predict the sentiment of a movie review

In [4]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten

from keras.layers.embeddings import Embedding

/Users/JC/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
import os
import sys
import numpy as np

In [3]:
from nltk.corpus import movie_reviews as reviews

X = [reviews.raw(fileid) for fileid in reviews.fileids()]
y = [reviews.categories(fileid)[0] for fileid in reviews.fileids()]

In [8]:
len(X[6])

3365

In [9]:
t = Tokenizer()
t.fit_on_texts(X)
vocab_size = len(t.word_index) + 1

In [10]:
encoded_docs = t.texts_to_sequences(X)

In [17]:
#print(encoded_docs[1])

In [11]:
max_length = 4000
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [12]:
embeddings_index = dict()

In [13]:
f = open('glove.6B.200d.txt')

In [14]:
from numpy import array
from numpy import asarray
from numpy import zeros
for line in f: 
    value = line.split()
    word = value[0]
    coefs = asarray(value[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()


In [15]:
embedding_matrix = zeros((vocab_size, 200))

In [16]:
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [18]:
e = Embedding(vocab_size, 200, weights=[embedding_matrix], input_length=4000, trainable=False)

In [19]:
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [20]:
model = Sequential()
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [21]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [22]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 4000, 200)         8659400   
_________________________________________________________________
flatten_1 (Flatten)          (None, 800000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 800001    
Total params: 9,459,401
Trainable params: 800,001
Non-trainable params: 8,659,400
_________________________________________________________________
None


In [24]:
import numpy as np
myarray = np.asarray(y)

In [25]:
y_array_t = []
for i in range(0,len(myarray)):
    if myarray[i] == "neg":
        l = 0
    if myarray[i] == "pos":
        l = 1
    y_array_t.append(l)

In [26]:
a_array = np.asarray(y_array_t)

In [27]:
#finally, I use this a_array as my y_train
a_array

array([0, 0, 0, ..., 1, 1, 1])

In [28]:
model.fit(padded_docs, a_array, epochs=20, verbose=0)

In [29]:
loss, accuracy = model.evaluate(padded_docs, a_array, verbose=0)

In [30]:
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


#training accuracy is 100%